In [ ]:
import ROOT as R

import uproot
import awkward as ak
import numpy as np
import pandas as pd
import numba as nb
import matplotlib.pyplot as plt

import time
import gc
import yaml
import json
from glob import glob
# from memory_profiler import profile

In [2]:
%load_ext memory_profiler
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

### loading data

In [3]:
file_name = 'data/ShuffleMergeSpectral_2.root'
tree_name = 'taus'

In [4]:
cache = uproot.LRUArrayCache("1 GB")
taus_lazy = uproot.lazy(f'{file_name}:{tree_name}', array_cache=cache)

In [5]:
taus_lazy

<Array [{run: 1, lumi: 118, ... 1]}] type='44436 * {"run": uint32, "lumi": uint3...'>

In [6]:
cache

<LRUArrayCache (1368636/1000000000 bytes full) at 0x2ae2451558e0>

In [7]:
cache.clear()

In [8]:
cache

<LRUArrayCache (0/1000000000 bytes full) at 0x2ae2451558e0>

### loading data [2]

In [9]:
t = uproot.open(f'{file_name}:{tree_name}')
taus_zip = t.arrays(how='zip')

In [10]:
# for taus in batch_yielder:
#     break

In [11]:
# taus

### loading data [3]

In [12]:
taus_r = R.RDataFrame(tree_name, file_name)

### benchmark means/std

#### RDataFrame

In [13]:
%%timeit
pt_mean = taus_r.Mean('pfCand_pt')
pt_mean.GetValue()

658 ms ± 18.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
pt_std = taus_r.StdDev('pfCand_pt')
pt_std.GetValue()

656 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### lazy awkward

In [15]:
%%timeit
ak.mean(taus_lazy['pfCand_pt'])

657 µs ± 41.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
ak.std(taus_lazy['pfCand_pt'])

17.5 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### zip awkward

In [17]:
%%timeit
ak.mean(taus_zip['pfCand', 'pt'])

508 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
%%timeit
ak.std(taus_zip['pfCand', 'pt'])

17.5 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
ak.std(ak.flatten(taus_zip['pfCand', 'pt']))

16.7 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### self-made awkward

In [20]:
%%timeit
pt_mean = ak.mean(taus_lazy['pfCand_pt'])
pt_std = np.sqrt(ak.sum((taus_lazy['pfCand_pt'] - pt_mean)**2) / ak.count(taus_lazy['pfCand_pt']))

17.7 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### compare S&M files

In [5]:
taus_1 = uproot.open(f'data/ShuffleMergeSpectral_1.root:{tree_name}').arrays(how='zip')

In [6]:
taus_3 = uproot.open(f'data/ShuffleMergeSpectral_3.root:{tree_name}').arrays(how='zip')

In [18]:
ak.count(taus_1), ak.count(taus_3)

(296449347, 283203975)

In [20]:
len(taus_1), len(taus_3)

(191700, 183251)

In [15]:
ak.mean(taus_1['pfCand', 'pt']), ak.mean(taus_3['pfCand', 'pt'])

(6.531693611959568, 6.531911499813611)

In [16]:
ak.std(taus_1['pfCand', 'pt']), ak.std(taus_3['pfCand', 'pt'])

(35.57811990694886, 35.44534597127814)

In [21]:
ak.mean(taus_1['pfCand', 'eta']), ak.mean(taus_3['pfCand', 'eta'])

(-0.0028388169881038566, -0.0022471090837211266)

In [22]:
ak.std(taus_1['pfCand', 'eta']), ak.std(taus_3['pfCand', 'eta'])

(1.259707597657328, 1.2584580628293074)

### dev

In [21]:
tree_name = 'taus'
var_structure = {}

In [22]:
file_names = glob('data/ShuffleMergeSpectral*.root')
file_names

['data/ShuffleMergeSpectral_5.root',
 'data/ShuffleMergeSpectral_4.root',
 'data/ShuffleMergeSpectral_3.root',
 'data/ShuffleMergeSpectral_1.root',
 'data/ShuffleMergeSpectral_2.root']

#### global vars

In [23]:
tau_dxy_valid = '(tau_dxy > -10) & (tau_dxy_error > 0)'
tau_dz_sig_valid = 'tau_dz_error > 0'
tau_ip3d_valid = '(tau_ip3d > - 10) & (tau_ip3d_error > 0)'

In [24]:
var_structure['global'] = {
###  "pt": ("pt > 50", {"pt": "sqrt(px**2 + py**2)"} ) <--- var_name: (selection, aliases) ###
    'npv': (None, None), 'rho': (None, None),
    'pv_x': (None, None), 'pv_y': (None, None), 'pv_z': (None, None), 'pv_chi2': (None, None), 'pv_ndof': (None, None),
    'tau_mass': (None, None),  'chargedIsoPtSum': (None, None), 'footprintCorrection': (None, None), 'neutralIsoPtSum': (None, None), 
    'photonPtSumOutsideSignalCone': (None, None), 'puCorrPtSum': (None, None), 
    
    'tau_dxy_pca_x': (None, None), 'tau_dxy_pca_y': (None, None), 'tau_dxy_pca_z': (None, None), 
    'tau_dxy': (tau_dxy_valid, None), 'tau_dxy_sig': (tau_dxy_valid, {'tau_dxy_sig': 'abs(tau_dxy)/tau_dxy_error'}),
    'tau_dz': (None, None), 'tau_dz_sig': (tau_dz_sig_valid, {'tau_dz_sig': 'abs(tau_dz)/tau_dz_error'}),
    'tau_ip3d': (tau_ip3d_valid, None), 'tau_ip3d_sig': (tau_ip3d_valid, {'tau_ip3d_sig': 'abs(tau_ip3d)/tau_ip3d_error'}),
    
    'tau_flightLength_x': (None, None), 'tau_flightLength_y': (None, None), 'tau_flightLength_z': (None, None), 'tau_flightLength_sig': (None, None),
    'tau_pt_weighted_dr_signal': (None, None), 'tau_leadingTrackNormChi2': (None, None),
    'tau_n_photons': (None, None),
    'leadChargedCand_etaAtEcalEntrance_minus_tau_eta': (None, {'leadChargedCand_etaAtEcalEntrance_minus_tau_eta': 'leadChargedCand_etaAtEcalEntrance - tau_eta'}),
}

#### ele vars

In [25]:
cc_valid = 'ele_cc_ele_energy >= 0'
has_closestCtfTrack = 'ele_closestCtfTrack_normalizedChi2 >= 0'

In [26]:
var_structure['ele'] = {
###  "pt": ("pt > 50 and abs(eta) < 2.1", {"pt": "sqrt(px**2 + py**2)"} ) <--- var_name: (selection, aliases) ###

    'ele_rel_pt': (None, {'ele_rel_pt': 'ele_pt/tau_pt'}),
    'ele_cc_ele_rel_energy': (cc_valid, {'ele_cc_ele_rel_energy': 'ele_cc_ele_energy/ele_pt'}),
    'ele_cc_gamma_rel_energy': (cc_valid, {'ele_cc_gamma_rel_energy': 'ele_cc_gamma_energy/ele_cc_ele_energy'}),
    'ele_cc_n_gamma': (None, None),
    'ele_rel_trackMomentumAtVtx': (None, {'ele_rel_trackMomentumAtVtx': 'ele_trackMomentumAtVtx/ele_pt'}),
    'ele_rel_trackMomentumAtCalo': (None, {'ele_rel_trackMomentumAtCalo': 'ele_trackMomentumAtCalo/ele_pt'}),
    'ele_rel_trackMomentumOut': (None, {'ele_rel_trackMomentumOut': 'ele_trackMomentumOut/ele_pt'}),
    'ele_rel_trackMomentumAtEleClus': (None, {'ele_rel_trackMomentumAtEleClus': 'ele_trackMomentumAtEleClus/ele_pt'}),
    'ele_rel_trackMomentumAtVtxWithConstraint': (None, {'ele_rel_trackMomentumAtVtxWithConstraint': 'ele_trackMomentumAtVtxWithConstraint/ele_pt'}),
    'ele_rel_ecalEnergy': (None, {'ele_rel_ecalEnergy': 'ele_ecalEnergy/ele_pt'}),
    'ele_ecalEnergy_sig': (None, {'ele_ecalEnergy_sig': 'ele_ecalEnergy/ele_ecalEnergy_error'}),

    'ele_eSuperClusterOverP': (None, None), 'ele_eSeedClusterOverP': (None, None), 
    'ele_eSeedClusterOverPout': (None, None), 'ele_eEleClusterOverPout': (None, None),
    'ele_deltaEtaSuperClusterTrackAtVtx': (None, None), 'ele_deltaEtaSeedClusterTrackAtCalo': (None, None),
    'ele_deltaEtaEleClusterTrackAtCalo': (None, None), 'ele_deltaPhiEleClusterTrackAtCalo': (None, None),
    'ele_deltaPhiSuperClusterTrackAtVtx': (None, None), 'ele_deltaPhiSeedClusterTrackAtCalo': (None, None),
    
    'ele_mvaInput_sigmaEtaEta': (None, None), 'ele_mvaInput_hadEnergy': (None, None),
    'ele_gsfTrack_normalizedChi2': (None, None), 'ele_gsfTrack_numberOfValidHits': (None, None), 'ele_mvaInput_sigmaEtaEta': (None, None),
    
    'ele_rel_gsfTrack_pt': (None, {'ele_rel_gsfTrack_pt': 'ele_gsfTrack_pt/ele_pt'}),
    'ele_gsfTrack_pt_sig': (None, {'ele_gsfTrack_pt_sig': 'ele_gsfTrack_pt/ele_gsfTrack_pt_error'}),
    
    'ele_closestCtfTrack_normalizedChi2': (has_closestCtfTrack, None), 'ele_closestCtfTrack_numberOfValidHits': (has_closestCtfTrack, None), 
}

#### neutralHadron vars

In [27]:
is_neutralHadron = 'abs(pfCand_pdgId) == 130'

In [28]:
var_structure['PfCand_neutralHadron'] = {
###  "pt": ("pt > 50", {"pt": "sqrt(px**2 + py**2)"} ) <--- var_name: (selection, aliases) ###

    'pfCand_nHad_rel_pt': (is_neutralHadron, {'pfCand_nHad_rel_pt': 'pfCand_pt/tau_pt'}),
}

#### compute sums

In [29]:
# %%timeit
sums = {var_type: {var: np.zeros(len(file_names), dtype='float64') for var in var_dict.keys()} for var_type, var_dict in var_structure.items()}
sums2 = {var_type: {var: np.zeros(len(file_names), dtype='float64') for var in var_dict.keys()} for var_type, var_dict in var_structure.items()}
counts = {var_type: {var: np.zeros(len(file_names), dtype='int64') for var in var_dict.keys()} for var_type, var_dict in var_structure.items()}
program_starts = time.time()
last_file_done = program_starts
for i, file_name in enumerate(file_names):
    with uproot.open(file_name, array_cache='10 GB') as f:
        for var_type, var_dict in var_structure.items():
            for var, (selection, aliases) in var_dict.items():
                taus_batches = f[tree_name].iterate(var, cut=selection, aliases=aliases, step_size='1 GB') # , how='zip'
                for batch in taus_batches:
                    sums[var_type][var][i] += ak.sum(batch[var])
                    sums2[var_type][var][i] += ak.sum(batch[var]**2)
                    counts[var_type][var][i] += ak.count(batch[var])
    processed_file = time.time()
    print(f'processed {file_name} in {processed_file - last_file_done:.2f} s')
    last_file_done = processed_file

processed data/ShuffleMergeSpectral_5.root in 6.99 s
processed data/ShuffleMergeSpectral_4.root in 8.28 s
processed data/ShuffleMergeSpectral_3.root in 5.55 s
processed data/ShuffleMergeSpectral_1.root in 5.88 s
processed data/ShuffleMergeSpectral_2.root in 2.01 s


In [30]:
sums['global']

{'npv': array([6736688., 8099652., 5300815., 5546621., 1287167.]),
 'rho': array([4952134.5, 5951399. , 3892448. , 4074656.5,  946514.5]),
 'pv_x': array([2603.8046875 , 3152.41967773, 2048.78613281, 2153.41918945,
         498.01062012]),
 'pv_y': array([ 9588.64355469, 11516.95117188,  7544.53417969,  7892.27441406,
         1826.89196777]),
 'pv_z': array([5541.16162109, 8135.78027344, 6629.40185547, 7491.61523438,
        1687.21313477]),
 'pv_chi2': array([19925870. , 23958688. , 15674731. , 16422384. ,  3781319.5]),
 'pv_ndof': array([26618356. , 32036432. , 20954744. , 21958678. ,  5054356.5]),
 'tau_mass': array([151784.        , 182567.15625   , 119485.3359375 , 124684.2734375 ,
         28979.16601562]),
 'chargedIsoPtSum': array([5934923.5 , 7082429.  , 4658769.5 , 4883036.5 , 1087270.75]),
 'footprintCorrection': array([1337027.75   , 1581763.375  , 1042740.     , 1099172.     ,
         253387.46875]),
 'neutralIsoPtSum': array([6453830.5, 7659388.5, 5006643. , 5291715. , 

#### compute mean/std

In [78]:
def compute_mean(sums, counts, var_type, var, aggregate=True, *files_range):
    if aggregate:
        if files_range:
            if len(files_range) == 2:
                return sums[var_type][var][files_range[0]:files_range[1]].sum()/counts[var_type][var][files_range[0]:files_range[1]].sum()
            else:
                raise ValueError("window should have 2 values")
        else:
            return sums[var_type][var].sum()/counts[var_type][var].sum()
    else:
        return sums[var_type][var]/counts[var_type][var]

In [72]:
def compute_std(sums2, counts, means, var_type, var, aggregate=True):
    if aggregate:
        return np.sqrt(sums2[var_type][var].sum()/counts[var_type][var].sum() - means[var_type][var]**2)
    else:
        return np.sqrt(sums2[var_type][var]/counts[var_type][var] - means[var_type][var]**2)

In [73]:
var_type = 'ele'
means, stds = {}, {}
means[var_type] = {var: compute_mean(sums, counts, var_type, var, True) for var in var_structure[var_type].keys()}
stds[var_type] = {var: compute_std(sums2, counts, means, var_type, var, True) for var in var_structure[var_type].keys()}

In [74]:
means[var_type]

{'ele_rel_pt': 0.916113256701234,
 'ele_cc_ele_rel_energy': 1.8027933535761105,
 'ele_cc_gamma_rel_energy': 0.11879583984841108,
 'ele_cc_n_gamma': -12.331005388542257,
 'ele_rel_trackMomentumAtVtx': 1.9692697477323486,
 'ele_rel_trackMomentumAtCalo': 1.9693049342969273,
 'ele_rel_trackMomentumOut': 0.8856736523810138,
 'ele_rel_trackMomentumAtEleClus': 0.8856774228196869,
 'ele_rel_trackMomentumAtVtxWithConstraint': 1.713368243273185,
 'ele_rel_ecalEnergy': 2.0425174555574177,
 'ele_ecalEnergy_sig': 71.1713875496313,
 'ele_eSuperClusterOverP': 2.567563566952315,
 'ele_eSeedClusterOverP': 2.1962159953011398,
 'ele_eSeedClusterOverPout': 6.795766402906332,
 'ele_eEleClusterOverPout': 4.282800449703179,
 'ele_deltaEtaSuperClusterTrackAtVtx': 7.846321923490073e-05,
 'ele_deltaEtaSeedClusterTrackAtCalo': -0.0002575860463610282,
 'ele_deltaEtaEleClusterTrackAtCalo': -0.00026019224681400843,
 'ele_deltaPhiEleClusterTrackAtCalo': 2.2400793527392578e-05,
 'ele_deltaPhiSuperClusterTrackAtVtx': 

In [75]:
stds[var_type]

{'ele_rel_pt': 0.7443117810719942,
 'ele_cc_ele_rel_energy': 1.3071761376004307,
 'ele_cc_gamma_rel_energy': 0.28672307489626236,
 'ele_cc_n_gamma': 118.31283353581435,
 'ele_rel_trackMomentumAtVtx': 91.06625562185117,
 'ele_rel_trackMomentumAtCalo': 91.06625494278842,
 'ele_rel_trackMomentumOut': 12.725610212405734,
 'ele_rel_trackMomentumAtEleClus': 12.725610029133072,
 'ele_rel_trackMomentumAtVtxWithConstraint': 7.802024362485641,
 'ele_rel_ecalEnergy': 1.3705716876132157,
 'ele_ecalEnergy_sig': 54.990134165503086,
 'ele_eSuperClusterOverP': 13.974667084820739,
 'ele_eSeedClusterOverP': 12.79989999732592,
 'ele_eSeedClusterOverPout': 32.436279192279365,
 'ele_eEleClusterOverPout': 16.40539400357059,
 'ele_deltaEtaSuperClusterTrackAtVtx': 0.044193842817713214,
 'ele_deltaEtaSeedClusterTrackAtCalo': 0.04895207792574902,
 'ele_deltaEtaEleClusterTrackAtCalo': 0.050599550377552406,
 'ele_deltaPhiEleClusterTrackAtCalo': 0.053579564561007105,
 'ele_deltaPhiSuperClusterTrackAtVtx': 0.052433

#### dependance on files number

In [86]:
# sums['global']['npv']/counts['global']['npv']
sums['ele']['ele_rel_pt']/counts['ele']['ele_rel_pt']

array([0.91678451, 0.92106623, 0.91003978, 0.91399554, 0.91564447])

In [84]:
compute_mean(sums, counts, 'ele', 'ele_rel_pt', True, 0, -1)

0.9161368333869275

In [85]:
compute_mean(sums, counts, 'ele', 'ele_rel_pt', True)

0.916113256701234

#### validate

In [50]:
# t = uproot.open(f'{file_names[0]}:{tree_name}')
# taus = t.arrays(how='zip')

In [51]:
cache = uproot.LRUArrayCache("1 GB")
taus = uproot.lazy(file_names, array_cache=cache)

In [55]:
var_type = 'ele'
var = 'ele_eSeedClusterOverP'

In [56]:
ak.mean(taus[var]), ak.std(taus[var])

(2.196216096014026, 12.799899045004858)

In [57]:
means[var_type][var], stds[var_type][var]

(2.1962159953011398, 12.79989999732592)

#### write to json

In [47]:
means_dict = {}
stds_dict = {}
for var_type in var_structure.keys():
    means_dict[var_type] = {var: compute_mean(sums, counts, var_type, var, True) for var in var_structure[var_type].keys()}
    stds_dict[var_type] = {var: compute_std(sums2, counts, means_dict, var_type, var, True) for var in var_structure[var_type].keys()}

In [48]:
with open('means.json', 'w') as fout:    
    json.dump(means_dict, fout)

In [49]:
with open('stds.json', 'w') as fout:    
    json.dump(stds_dict, fout)